In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
u = []
responses = []
start = 1;
end = 400
url = 'https://www.commonfloor.com/mumbai-property/for-sale?page=' #Getting data around 29754
for i in range(start,end):  # using 10000(should be enough ig)
    u.append(url+str(i))

In [4]:
responses = []
for i in range(start,end):    
    responses.append(requests.get(u[i-1]))
docs = []
for i in range(start,end):
    docs.append(BeautifulSoup(responses[i-1].text,'html.parser'))
containers = []
for i in range(start,end):
    container = docs[i-1].find_all('div',{'class': 'st_title'})
    containers.append(container)    
    

In [8]:
base_url = 'https://www.commonfloor.com/'

urls = []
for i in range(start,end):
    for j in range(len(containers[i-1])):
        links = container[j].find_all('a',href=True)
        link = container[j].find('a',href=True)
        urls.append(base_url + link['href'])
    

In [10]:
df = pd.DataFrame(urls)
df.to_csv('urls.csv',index=None)

In [11]:
import requests
from bs4 import BeautifulSoup
import re


In [ ]:
REQUEST_THRESHOLD = 1500
PAUSE_DURATION = 60
request_counter = 0

responses = []
# urls = [
#     'https://www.commonfloor.com//listing/-1-bhk-apartment-for-sale-in-kalyan-east-mumbai-at-ahmahf-empire/aqet8jdqwo80shmn',
#     'https://www.commonfloor.com//listing/not-furnished-1-bhk-apartment-for-sale-in-chandivali-mumbai-at-sumer-life-casa/ebo3klfnw98wcvwh',
#     'https://www.commonfloor.com//listing/-1-bhk-apartment-for-sale-in-andheri-west-mumbai-at-residential-apartment/5z2ghzlxc4g17130'

# ]
titles = []
docs = []
areas = []
prices = []
bedrooms_list = []
bathrooms_list = []
builtup_area_list = []
carpet_area_list = []
condition_list = []
# subtitle_dvi[] = []

# getting the html docs
for i in range(len(urls)):
    response = requests.get(urls[i])
    responses.append(response)
    doc = BeautifulSoup(responses[i].text,'html.parser')
    # docs.append(doc)
    # doc = BeautifulSoup(res.text,'html.parser')

    # Scraping datas from docs
    t_tags = doc.find('h1')
    title = t_tags.text.split('Project: ')[0].strip()
    titles.append(title)
    titles[i] = re.sub(r'\s+', ' ', titles[i]).strip()

    
    subtitle_div = doc.find('div',{'class': '_headsubtitle'})
    if subtitle_div:
        a_elements = subtitle_div.find_all('a', href=True)
        
        if(len(a_elements)==2):
            area = a_elements[1].text
        else:
            area = a_elements[0].text
        # locations.append(location)
        areas.append(area)
    # Getting the prices, builluparea(not carpet_area)
    price_tags = doc.find('div',{'class': 'subfeatures'})
    if price_tags:
        price = price_tags.span.text.strip()
        prices.append(price)
        
    area_tags = doc.find('ul',{'class': '_33'})
    if area_tags:
        builtup_area = area_tags.contents[1].span.text
        builtup_area_list.append(builtup_area)
        carpet_area = area_tags.contents[3].span.text
        carpet_area_list.append(carpet_area)
        bedrooms = area_tags.contents[7].span.text
        bedrooms_list.append(bedrooms)
        bathrooms = area_tags.contents[5].span.text
        bathrooms_list.append(bathrooms)
        condition = area_tags.contents[9].span.text
        # condition = area_tags.contents[7].span.text
        condition_list.append(condition)
    request_counter+=1
    
    # Checking for threshold
    if request_counter == REQUEST_THRESHOLD:
        printf("Pausing for {PAUSE_DURATION}")
        time.sleep(PAUSE_DURATION)

    #resetting the counter
    request_counter=0 
    

In [134]:
# df = pd.DataFrame(areas,prices,bedrooms_list,bathrooms_list,builtup_area_list)
df = pd.DataFrame({
    'title': titles,
    'location': areas,
    'bedroooms': bedrooms_list,
    'bathrooms': bathrooms_list,
    'carpet Area': carpet_area,
    'Built Up Area': builtup_area_list,
})

In [135]:
json_data=df.to_json(orient='records')
file_path = 'output.json'
with open(file_path, 'w') as file:
    file.write(json_data)

print(f'Data saved to {file_path}')



Data saved to output.json
